# ADR (Anomaly Detection by workflow Relations)

Software systems usually generate logs for troubleshooting. The logs are text messages recording system events, which can help engineers determine the system's runtime status. 

ADR employs matrix nullspace to mine numerical relations from log data and the mined relations can be used for both offline and online anomaly detection and facilitate fault diagnosis. ADR starts by parsing the raw logs into structured log events. Then the parsed log events are grouped into sequences by sessions or sliding windows. For each session or window, the occurrences of its log events are counted, resulting in the event-count-matrix. Then the available numerical relations are extracted from the event-count-matrix by evaluating its nullspace. Finally, the extracted relations are used to detect abnormal log sequences in the offline or online manner.

In the following parts, we use the BGL logs as example to show the capability of ADR.

For ease of presentation, the raw BGL logs are already parsed into structured log events and saved in _data/Drain_result_ (The raw data is parsed by Drain <sup>[1]</sup>)

## Offline anomaly detection at session level

In [4]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

from ADR.preprocess import *
from ADR.ADR import *

log_path = Path(r'data\Drain_result\bgl_10k\ECM_sessions_10k.npz')
X_Y = np.load(log_path)
x, y = X_Y['df_X'], X_Y['df_Y']
train_size = 361
x_train, y_train, x_test, y_test = split_to_train_test_by_num(x, y, train_size)

model = ADR(1, add_parity=True, add_tf=True)
model.fit(x_train, y_train)

print("=====BGL dataset=====")
print('precision, recall, f1, mcc for testing set:')
model.evaluate(x_test, y_test)

=====BGL dataset=====
precision, recall, f1, mcc for testing set:


(0.9408, 1.0, 0.9695, 0.9372821480127299)

## Online anomaly detection at entry level

### (1) Use PSO to find the optimal window size

In [ ]:
%reload_ext autoreload
%autoreload 2

from ADR import pso_window_size
pso_window_size.main()

### (2) Split the logs to windows and detect anomalies

In [3]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
import numpy as np

bgl_sample_path = Path(r'data\Drain_result\bgl_10k\BGL.log_structured500k.csv')
bgl_template_path = Path(r'data\Drain_result\bgl_10k\BGL.log_templates.csv')

df_bgl_sample = pd.read_csv(bgl_sample_path, sep=',', header=0)
df_bgl_sample["bLabel"] = 1
df_bgl_sample.loc[df_bgl_sample["Label"]=="-", "bLabel"] = 0

# split the logs by number windows
from ADR.ADR_Online import *

list_EventIds = pd.read_csv(bgl_template_path, header=0)["EventId"].to_list()
df_logs = df_bgl_sample.iloc[0:100000, :]

train_ratio = 0.7
num_train = int(df_logs.shape[0] * train_ratio)

df_bgl_train = df_logs.iloc[0:num_train, :]
df_bgl_test = df_logs.iloc[num_train:, :]

window_size = 53
print('='*10)
print(f'window size is {window_size}')
model = ADR_Online(window_size=window_size, highest_degree=1, add_parity=True, add_tf=True)
print('fitting the model...')
print('(this step may take long, but it only needs to be run one time to obtain the relations that are used for following anomaly detection)')
model.fit(df_bgl_train, list_EventIds)
model.evaluate(df_bgl_test)

window size is 53
fitting the model...
(this step may take long, but it only needs to be run one time to obtain the relations that are used for following anomaly detection)
grouping logs to windows...
anomaly detection...


ValueError: shapes (0,1) and (1153,1139) not aligned: 1 (dim 1) != 1153 (dim 0)

## References

[1] P. He, J. Zhu, Z. Zheng, and M. R. Lyu, “Drain: An Online Log Parsing Approach with Fixed Depth Tree,” in 2017 IEEE International Conference on Web Services (ICWS), Jun. 2017, pp. 33–40, doi: 10.1109/ICWS.2017.13.